# This is a work-in-progress notebook

We wish to know this:

1. How well does the model identify the correct number of senses for the target word?
2. **How well does the model identify the correct senses for the target word?**
3. **How well does the model assign the right words to a given sense of the target word?**
4. How well does the model assign the senses to the time intervals for the target word?

The script will evaluate **Q2** and **Q3**. Q4 will follow.

In [150]:
# Basic variables and imports:

import codecs, csv, os, time, re, io
from os import listdir
from os.path import isfile, join

# directories
dir_in = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_input"))
dir_out = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_output"))


s_senses = io.open(dir_in+"/senses_69419.txt","r")
k_senses = io.open(dir_in+"/mus.dat","r")

# DEBUG:
#s_senses = io.open(dir_in+"/senses_69419_debug.txt","r")
#k_senses = io.open(dir_in+"/mus_debug.dat","r")
# k0 = mus4
# k1 = mus3
# k2 = mus2
# k3 = mus1
# k4 = nothing

file_senses = s_senses.readlines()[1:]
output_senses = k_senses.read()

## TODO

- ~~create the notebook~~
- ~~organise the notebook~~
- ~~write "general idea" pseudocode for the evaluation~~
- ~~get input files~~
- ~~figure out data structures to store the variables~~
- ~~write actual code~~

## Q2: How well does the model identify the correct senses for the target word?

### Pseudocode 

In [151]:
# For each target word, we have a list of senses  s (given by the expert)
# For each target word, we have a list of senses k (given by the model)
# This Q consists in matching s and k, and doing so in a confident way --> confidence score

for each k:
    for each s:
        create conf(k,s)

# What is conf(k,s)?
        conf(k,s) = (p1*match(w1,s)+p2*match(w1,s)+px(wx,s))/10 WHERE
    
            px = probability of word wx 
                
                and
            
            match(wx,s) =   1/number_of_senses_assigned_to_wx if s_is_one_of_them 
            
                    or 
                            0 if w_is_not_associated_to_s
                
# Once we have gone through all s for one k, we have to choose the best k for s. How? (TBD, cfr Valerio and Barbara)

# Once all ks have been assigned to all ss (or NA), we can calculate a general confidence score for the model.
# One easy way to do that: 

conf_score_model = number_of_non_NA/k

SyntaxError: invalid syntax (<ipython-input-151-82fa54601c9a>, line 5)

### Real code

Steps:

- extract all senses from the file
- use those senses as keys for a dictionary, `dict_of_words`
- fill the dictionary: for each key, we store a list of words pertaining to that sense
- transform the lists as sets so as to remove duplicates within the same sense
- create a dictionary with a word as a key and its weight as a value, depending on how many senses it appears
- parse the model output and get the probability weights for each word
- do not take into account the first line
- take care of empty lines

Todo:



In [152]:
expert_senses = list() # list where we store all sense ids provided by expert
for s in file_senses: # 60 for testing purposes
    s = s.split("\t")
    sense = s[11] # The sense ID is after the 10th tab
    expert_senses.append(sense)
    
#print(len(expert_senses),expert_senses,len(set(expert_senses)))

expert_senses = list(set(expert_senses)) # we only keep the unique senses
number_of_s = len(expert_senses)  # we create a variable that stores the number of unique senses
print("Number of senses:",number_of_s)

# This dictionary has a sense as a key, and a list of words as a value. 
dict_of_words = dict()
# This list stores all words
list_of_all_words = list()
# This dictionary stores all words as keys and their weight as value
word_weight = dict()

for i in range(0,number_of_s): # for each sense, we create a dictionary entry which has a list as value
    dict_of_words[expert_senses[i]] = list()

    for s in file_senses: # we go back in the file
        s = s.split("\t") # splitting on tabs
        
        sentence_of_ids = s[8] # 8 is for IDs, 9 is for words
        list_of_ids = sentence_of_ids.split(" ")  # splitting on spaces
        for word_id in list_of_ids:
            if s[11] == expert_senses[i]:      # we store all words for one sense 
                dict_of_words[expert_senses[i]].append(word_id)
            list_of_all_words.append(word_id) # we store all words, we'll iterate over that for scores
        

    # Here, we remove duplicates
    #dict_of_words[expert_senses[i]].append("79223") #testing
    dict_of_words[expert_senses[i]] = set(dict_of_words[expert_senses[i]]) 
      
    print(expert_senses[i],set(dict_of_words[expert_senses[i]]))
    print("\n\n")


Number of senses: 4
mus-1 {'16430', '54812', '18937', '7944', '5521', '21812', '12641', '112472', '76335', '42890', '81343', '101716', '24003', '59708', '48867', '4670', '31562', '21431', '66746', 'nlsj47984', '23690', '3986', '34855', '50093', '38547', '104872', '23468', '95853', '74127', '105344', '94483', '71262', '5009', '21802', '68970', '12349', '113060', '75910', '19641', '403', '34603', '26197', 'nlsj80462', '108536', '11072', '107167', '25007', 'nlsj68228', '37851', '62205', '63814', '65565', '2224', '66173', '55997', '678', '17730', '75948', '92662', '84725', '80557', '21487', '79595', '5112', '52332', '42842', '97285', '37776', '24801', '27629', '31948', '53956', '106502', '74735', '80042', '17381', 'nlsj9526', '7498', '116059', '104355', '28360', '80664', '40545', '13427', '6174', '13039', 'nlsj98815', 'nlsj8979', '19268', '79223', '92406', '31491', '10056', '103972', '54113', '95074', '46176', '93449', '41502', '70105', '2219', '260', '15047', '26943', '42887', '51241', 'n

In [160]:
# For every word in the list of words that we have
# we count the number of senses it appears in
# we use that number to divide its importance: 1 sense = 1 importance; 2 senses = 0.5 importance
# this can be finetuned

for word in list_of_all_words:
    x = 0
    for i in range(0,number_of_s):
        if word in dict_of_words[expert_senses[i]]:
            x += 1 
        if x != 0:
            word_weight[word] = 1/x
        
    
    #print(word,word_weight[word])  

#### parsing output.dat
- split on "===============  per time  ===============" and keep first part
- transform that into a list, then
- get lines that start with "p(w|s)"
- count those, k = that number
- split the line on ":", keep the second part
- split the rest on ";", it's [ID] = prob_from_this_ID

In [217]:
lines_output = output_senses.split("===============  per time  ===============")[0].split("\n")

number_of_the_k = 0

k_words_with_prob = dict()

for line in lines_output:
    if line[:6] == "p(w|s)":
        line = line.split(":")[1]
        line = line.split(";")
        #print(number_of_the_k,line)
        dico_word_prob = dict()
        temp_dict = dict()
        k_words_with_prob[number_of_the_k] = list()
        
        line = line[:-1] # last item of the list is empty
        
        total_probability = 0 # to have relative probs
        for word_prob in line:
            

        
            #word_prob = word_prob.split(",")
            #for word in word_prob:
            probability = re.findall("([\d.\w]*)",word_prob)
            if probability:
                probability = list(filter(None,probability))
                    
            total_probability += float(probability[1])
            print("word id",probability[0],"; probability",probability[1])
        
            dico_word_prob[probability[0]] = float(probability[1])
        #print(type(k_words_with_prob[number_of_the_k]))
        
        for i in dico_word_prob.keys():
            
            temp_dict[i] = float(dico_word_prob[i]/total_probability)
            k_words_with_prob[number_of_the_k] = temp_dict
            
            print(k_words_with_prob[number_of_the_k])
            
        #k_words_with_prob[number_of_the_k] = [float(dico_word_prob[i]/total_probability) for i in dico_word_prob]
        #print(k_words_with_prob[number_of_the_k])
        print(temp_dict)
        number_of_the_k += 1
        



word id 28355 ; probability 0.088
word id 69419 ; probability 0.069
word id 57460 ; probability 0.056
word id 114587 ; probability 0.042
word id 42071 ; probability 0.041
word id 35267 ; probability 0.035
word id 51647 ; probability 0.035
word id 64448 ; probability 0.023
word id 45980 ; probability 0.018
word id 53826 ; probability 0.017
{'28355': 0.2075471698113207}
{'28355': 0.2075471698113207, '69419': 0.16273584905660377}
{'28355': 0.2075471698113207, '69419': 0.16273584905660377, '57460': 0.1320754716981132}
{'28355': 0.2075471698113207, '69419': 0.16273584905660377, '57460': 0.1320754716981132, '114587': 0.0990566037735849}
{'28355': 0.2075471698113207, '69419': 0.16273584905660377, '57460': 0.1320754716981132, '114587': 0.0990566037735849, '42071': 0.09669811320754716}
{'28355': 0.2075471698113207, '69419': 0.16273584905660377, '57460': 0.1320754716981132, '114587': 0.0990566037735849, '42071': 0.09669811320754716, '35267': 0.08254716981132075}
{'28355': 0.2075471698113207, '69

#### k_words_with_prob
This dictionary has the sense number 'k' as keys and the a dictionary of [word] = probability as values.
Example below.

In [205]:
#print("Probability for word ID 5390 in sense k = 4:",k_words_with_prob[4]["5390"])
print(type(k_words_with_prob[4]))

<class 'float'>


In [215]:
for key in k_words_with_prob.keys():
    print("output sense",key)
    for i in range(0,number_of_s):
        print("\texpert sense number ", i, expert_senses[i])
        for second_key in k_words_with_prob[key].keys(): # Barbara's note: shouldn't it be k_words_with_prob[i] here?
            print("\t\tword from annotation for sense", key, ":", second_key)
            if second_key in dict_of_words[expert_senses[i]]:
                print("\t\t\tword ", second_key, "is in output for sense", key, "with probability:", k_words_with_prob[key][second_key], "and weight:", word_weight[second_key])

                
# Here we get all the senses and for each sense we do a matching between the k words and s words and get the probability
# For some reason the first word for each sense arrives several times


output sense 0
	expert sense number  0 mus-1
		word from annotation for sense 0 : 28355
			word  28355 is in output for sense 0 with probability: 0.2075471698113207 and weight: 1.0
		word from annotation for sense 0 : 69419
			word  69419 is in output for sense 0 with probability: 0.16273584905660377 and weight: 0.25
		word from annotation for sense 0 : 57460
		word from annotation for sense 0 : 114587
			word  114587 is in output for sense 0 with probability: 0.0990566037735849 and weight: 1.0
		word from annotation for sense 0 : 42071
		word from annotation for sense 0 : 35267
			word  35267 is in output for sense 0 with probability: 0.08254716981132075 and weight: 1.0
		word from annotation for sense 0 : 51647
			word  51647 is in output for sense 0 with probability: 0.08254716981132075 and weight: 1.0
		word from annotation for sense 0 : 64448
			word  64448 is in output for sense 0 with probability: 0.05424528301886792 and weight: 1.0
		word from annotation for sense 0 : 45980
			

In [218]:
## Calculating confidence score for each (words_of_k,words_of_s) pair

# conf(k,s) = (p1*match(w1,s)+p2*match(w1,s)+px(wx,s))/10
        # match(wx,s) =   1/number_of_senses_assigned_to_wx if s_is_one_of_them 

##### TODO: for now conf[k,s] is multiplied by the number of expert senses --- FIX
    
print("number of expert senses s:",number_of_s)
print("number of model output senses k:",len(k_words_with_prob.keys()))
compteur = 0

match = dict()
conf = dict()
for k in k_words_with_prob.keys():  # for each output sense, we go through...
    print("\n")
    print("Choose best match for k =",k)
    for s in range(0,number_of_s):       # each expert sense
        
        conf[k,s] = 0 
        
        #print("expert sense",s)
        for mot in k_words_with_prob[k]:      # for each word within output by the model for the output sense
            #print(k,mot)
            #print(expert_senses[s])
            
            if mot in dict_of_words[expert_senses[s]]:  # if that word exists in the list of expert words for that sense
                
                #print(s,dict_of_words[expert_senses[s]])
                #print(k_words_with_prob[k][mot])
                
                for word in list_of_all_words:  # this help getting a key for a dictionary later on
                    if mot == word:
                        match_weighted = float((k_words_with_prob[k][mot]))*word_weight[word] #this dictionary cfr comment on line 24
                        # word_weight[word] is already "1/number_of_expert_senses_assigned_to_this_word"
                        
                        #print("sense",expert_senses[s],"word",word,"match_weighted",match_weighted)
                        
                        #print(k,s,conf[k,s])

                        
                        # To fix? 
                        # The way the code works is that all matches happen number_of_s times
                        # (number_of_s = number of expert senses)
                        # easy fix is to divide the match score by number_of_s
                        
                        conf[k,s] = conf[k,s] + match_weighted/4
                        
                    #else: 
                        #print(word,"has no match for sense",expert_senses[s])
                        #print(word,word_weight[word],"match",k_words_with_prob[k][mot],"match weighted",match_weighted)
                    #print("test1")
                #print("test2")
                
                    #compteur += 1
                
        if (k,s) in conf.keys():
        
            conf[k,s] = conf[k,s]/10  # 10 is hardcoded, that's the number of words the model outputs
            print("k =",k,"\t s =",s,"(= expert sense",expert_senses[s],")\t conf[k,s] =",conf[k,s])
            
            #print(compteur)
            
    #print(k_words_with_prob[k])


number of expert senses s: 4
number of model output senses k: 5


Choose best match for k = 0
k = 0 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.8360259433962197
k = 0 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.8100825471698055
k = 0 	 s = 2 (= expert sense w )	 conf[k,s] = 0.7647995283018816
k = 0 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 0.7671580188679193


Choose best match for k = 1
k = 1 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.03452054794520548
k = 1 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.019726027397260266
k = 1 	 s = 2 (= expert sense w )	 conf[k,s] = 0.05835616438356163
k = 1 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 0.06219178082191781


Choose best match for k = 2
k = 2 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.045098039215686246
k = 2 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.04779411764705884
k = 2 	 s = 2 (= expert sense w )	 conf[k,s] = 0.0
k = 2 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 0.2012254901960786


Choose best match for k

Barbara's note: "key" and "i" may be different senses; for example, for key=0, this is the first sense in the output, and i=0 is the first sense annotated by the expert. I think what we want here is to try matching all key values with all i values; for each (key, i) pair, we get the conf(key, i), as in pages 5 ff. of the Goals and plan.docx document.

Then, when possible, we can pick the best "i" for each "key"; we haven't yet decided how, but it will probably have to do with the maximum conf value.

Once we have a key-->i mapping, we can calculate precision and recall..

## Q3: How well does the model assign the right words to a given sense of the target word?

In [158]:
# For each k, we use the words given by the expert as unquestionable truth.
# Judging the model's assignment of words to a given sense becomes a question of precision and recall.

# precision is all correct w weighted by their respective probabilities / all w weighted by their probabilities

for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
        w_weight = p*1
        denominator += w_weight
    precision = numerator/denominator
    
# recall is all correct w weighted by their respective probabilities / all w assigned to the sense by the expert
for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
    denominator = len(expert_list)
    recall = numerator/denominator
    
# f-score can be used as well

for each k:
    f_score = 2 * precision * recall / (precision+recall)

SyntaxError: invalid syntax (<ipython-input-158-f315d04cacf9>, line 6)

In [164]:
# TODO: now that we have prob + word for each k,s pair we can calculate precision and recall 

list_with_ks = list()  # this list stores the k,s matches found above
list_with_ks = ["0,0","0,1","0,2","0,3","1,0","1,1","1,2","1,3","2,0","2,1","2,2","2,3","3,0","3,1","3,2","3,3","4,0","4,1","4,2","4,3"]


for key in list_with_ks:
    numerator_recall = 0
    denominator_precision = 0
    numerator_precision = 0
    for word in k_words_with_prob[int(key[0])]:    
        if word in dict_of_words[expert_senses[int(key[2])]]:    # this is bad but works for now
            w_weight_recall = k_words_with_prob[int(key[0])][word] * 1
            numerator_recall += float(w_weight_recall)
            w_weight_precision = k_words_with_prob[int(key[0])][word] * 1
            numerator_precision += float(w_weight_precision)
    
        denominator_precision += float(w_weight_precision)
    denominator_recall = len(dict_of_words[expert_senses[int(key[2])]])
    
            
    print("For pair ks",key,":")
    print("The RECALL is",numerator_recall,"/",denominator_recall,"=",numerator_recall/denominator_recall)
    print("The PRECISION is",numerator_precision,"/",denominator_precision,"=",numerator_precision/denominator_precision,"\n")
    if (numerator_precision/denominator_precision)+(numerator_recall/denominator_recall) != 0:
        print("The F-SCORE is", (2*(numerator_precision/denominator_precision)*(numerator_recall/denominator_recall)/((numerator_precision/denominator_precision)+(numerator_recall/denominator_recall))),"\n")
    else:
        print("No F-SCORE, can't divide by 0\n\n")

For pair ks 0,0 :
The RECALL is 0.31000000000000005 / 836 = 0.0003708133971291867
The PRECISION is 0.31000000000000005 / 0.43900000000000006 = 0.7061503416856493 

The F-SCORE is 0.0007412375558767587 

For pair ks 0,1 :
The RECALL is 0.166 / 337 = 0.0004925816023738873
The PRECISION is 0.166 / 0.44499999999999995 = 0.3730337078651686 

The F-SCORE is 0.0009838640371023428 

For pair ks 0,2 :
The RECALL is 0.08600000000000001 / 114 = 0.0007543859649122807
The PRECISION is 0.08600000000000001 / 0.61 = 0.14098360655737707 

The F-SCORE is 0.0015007416455806648 

For pair ks 0,3 :
The RECALL is 0.08700000000000001 / 225 = 0.0003866666666666667
The PRECISION is 0.08700000000000001 / 0.536 = 0.1623134328358209 

The F-SCORE is 0.0007714954597048809 

For pair ks 1,0 :
The RECALL is 0.063 / 836 = 7.535885167464115e-05
The PRECISION is 0.063 / 0.6299999999999999 = 0.10000000000000002 

The F-SCORE is 0.00015060420974624385 

For pair ks 1,1 :
The RECALL is 0.036 / 337 = 0.00010682492581602373

In [59]:
print(k_words_with_prob)

{0: {'28355': '0.088', '69419': '0.069', '57460': '0.056', '114587': '0.042', '42071': '0.041', '35267': '0.035', '51647': '0.035', '64448': '0.023', '45980': '0.018', '53826': '0.017'}, 1: {'79223': '0.063', '92927': '0.057', '46574': '0.038', '67660': '0.038', '103085': '0.036', '86112': '0.030', '101982': '0.029', '75808': '0.026', '68539': '0.024', '54607': '0.024'}, 2: {'62258': '0.080', '64586': '0.052', '58271': '0.048', 'nlsj86871': '0.041', '101851': '0.039', 'nlsj183': '0.037', '75653': '0.031', 'nlsj59923': '0.030', '70105': '0.026', '82758': '0.024'}, 3: {'102000': '0.035', '70495': '0.035', '7182': '0.029', '70958': '0.024', '49589': '0.021', '83174': '0.021', '106676': '0.019', '22209': '0.018', '93388': '0.017', '16132': '0.016'}, 4: {'nlsj5634': '0.039', '61925': '0.037', '12620': '0.030', '69419': '0.027', '104421': '0.027', '91085': '0.024', '71308': '0.022', '115748': '0.021', '19641': '0.020', '5390': '0.019'}}


In [165]:
print(expert_senses)

['mus-1', 'mus-4', 'w', 'mus-2']


In [169]:
for i in expert_senses:
    #print(i,dict_of_words[i])

SyntaxError: unexpected EOF while parsing (<ipython-input-169-d6839a6dbd08>, line 2)

## Qx: Model(s) comparison against annotated subcorpus (sense importance evolution + sense emergence)

### Pseudocode

1. Parse senses_target.txt to get:

    1.1 the date
    
    1.2 the number of senses at that date
    
    1.3 the number of uses of each sense at that date
    
    
2. Using the numbers found in 1.3, plot the emergence of new senses and the distribution of others


confidence interval!
